In [1]:
import andi
import numpy as np
import csv as csv
import numpy as np

from sklearn.metrics import mean_absolute_error
import tensorflow as tf



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import losses, metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import LSTM

from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.models import load_model
import os
from matplotlib import pyplot as plt
from data_split import data_split
from many_net import many_net, many_net_diffv, many_net_only_diff

Using TensorFlow backend.


In [2]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv1D, Dropout, BatchNormalization, Flatten

from keras.regularizers import l2 as regularizer_l2

from keras.optimizers import Adam
#from keras.utlils.np_utils import to_categorical

In [3]:
os.getcwd()

'/Users/stebo/andi_num/ANDI_datasets-master'

Importing the data

In [4]:
import csv
trajs_from_files = csv.reader(open('validation_for_scoring/task1.txt','r'), delimiter=';', 
                                        lineterminator='\n',quoting=csv.QUOTE_NONNUMERIC)
validation = [[],[],[]]
for trajs in enumerate(trajs_from_files):
    validation[int(trajs[1][0])-1].append(trajs[1][1:])

Importing the networks

In [5]:
model_norm_long2=load_model('task1_1d_norm_long2.h5')

model_norm_chi=load_model('task1_1d_v0_norm_chi.h5')

model_norm_short=tf.keras.models.load_model('Model_1D_recdout_125.h5')

model_norm_short2=load_model('task1_1d_v0_norm_short.h5')



In [6]:
meta_model=[model_norm_short, model_norm_chi, model_norm_long2]

If some of the networks take the diff and some not use many_net_diffv. Recall that di is the array of dimensions of input of different nets

In [8]:
predictions=many_net_diffv(meta_model,validation[0],diffv=[True,False,False])

If none of the networks require the diff use

In [ ]:
predictions=many_net(meta_model,validation[0])

If all of the networks require the diff use

In [ ]:
predictions=many_net_only_diff(meta_model,validation[0])

In [9]:
pred_to_txt = np.ones((len(predictions), 2))
pred_to_txt[:, 1] = predictions

np.savetxt('task1.txt', pred_to_txt.astype(float), fmt = '%1.5f', delimiter = ';')